In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import os
os.chdir('/content/drive/My Drive/project/')
!ls

dataset			   LR_training.ipynb   MNB.ipynb
Dataset_Exploration.ipynb  LSTM_train.ipynb    models
dataset_preprocess.ipynb   ML_Pipelines.ipynb  outputs


In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [42]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Loading Machine Learning Model

In [43]:
import pickle
model_path = "models/LR_model.sav"
model = pickle.load(open(model_path, 'rb'))

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [63]:
# Loading Tfidf Vector from saved models
# https://www.kaggle.com/mattwills8/fit-transform-and-save-tfidfvectorizer
# https://stackoverflow.com/questions/29788047/keep-tfidf-result-for-predicting-new-content-using-scikit-for-python


from sklearn.feature_extraction.text import TfidfVectorizer

import joblib
tfidf_path = "models/tfidf_vec.pkl"
tfidf_vec = joblib.load(tfidf_path)

tf1_new = TfidfVectorizer(analyzer='word', ngram_range=(1,3), stop_words = "english", lowercase = True
                          , vocabulary = tfidf_vec.vocabulary_)

ERROR! Session/line number was not unique in database. History logging moved to new session 60


In [0]:
from sklearn.base import TransformerMixin, BaseEstimator
import re
import nltk 
from nltk import WordNetLemmatizer
import multiprocessing as mp


class TextPreprocessor(BaseEstimator, TransformerMixin):

  def __init__(self, variety = 'BrE', user_abbrevs={}, n_jobs=1):
    self.variety = variety
    self.user_abbrevs = user_abbrevs
    self.n_jobs = n_jobs
  
  def fit(self, X, y = None):
    return self
  
  def transform(self, X, *_):
    # incomplete will be completed later
    # X_copy = X.copy()
    X_copy = X
    partitions = 1
    cores = mp.cpu_count()

    if self.n_jobs <= -1:
      partition = cores
    elif self.n_jobs <= 0:
      return X_copy.apply(slef._preprocess_text)
    else:
      partitions = min(self.n_jobs, cores)
    
    data_split = np.array_split(X_copy,partitions)
    pool = mp.Pool(cores)
    data = pd.concat(pool.map(self._preprocess_part, data_split))
    pool.close()
    pool.join()

    return data
  
  def _preprocess_part(self, part):
    return part.apply(self._preprocess_text)
  
  def _preprocess_text(self, text):
    hashtags = re.findall('#\w*',text)
    users = re.findall('@\w*',text)
    links = re.findall('(https|http)?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', text, flags=re.MULTILINE)
    links = list(set(links))
    # Removing Links
    text =  re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', ' ', text, flags=re.MULTILINE)
    # Removing mentions and hashtags
    text = re.sub(r"#(\w+)", ' ', text, flags=re.MULTILINE)
    text = re.sub(r"@(\w+)", ' ', text, flags=re.MULTILINE)
    # Removing punctuations
    text = re.sub(r'[^\w\d\s]', ' ', text)
    # convert to lower case
    text = re.sub(r'^\s+|\s+?$', ' ', text.lower())
    # Removing digits
    text = re.sub(r'\d', ' ', text)
    # Removing other symbols
    text = re.sub('[ãâªð³ÂÃÃ±¤¡¥¶¦§_®¯¹¾²µ½¼º]+', ' ', text)
    # collapse all white spaces
    text = re.sub(r'\s+', ' ', text)
    # text = re.sub('[Ã]', ' ', str(text))
    # remove stop words and perform stemming
    stop_words = nltk.corpus.stopwords.words('english')
    # 
    lemmatizer = WordNetLemmatizer()
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words)

    return [hashtags, users, links, [text]]

  

  def predict(self, text):
    tfidf_text = tfidf_vec.fit_transform(text)

    return model.predict(tfidf_text)[:,1]


In [0]:
text = "like this if you are a tribe fan"
# text = pd.Series(text)
# print(text)
processing = TextPreprocessor()

In [83]:
output = processing._preprocess_text(text)
output

[[], [], [], ['like tribe fan']]

In [0]:
tfidf_text = tf1_new.fit_transform([text])

In [68]:
print(tfidf_text)

  (0, 3985962)	0.5773502691896258
  (0, 2963558)	0.5773502691896258
  (0, 1103176)	0.5773502691896258


In [84]:
x = processing.predict(output[-1])
x

ValueError: ignored